# IMNKさんのベースラインを参考に
https://signate.jp/competitions/908/discussions/lightgbm-4?comment_id=6220#6220

In [ ]:
# driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# モジュールのインポート
import os
import copy
import pandas as pd
import numpy as np
import math
import pickle
import lightgbm as lgb
import itertools as it
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler

In [ ]:
# 作業用ディレクトリ
os.chdir('/content/drive/MyDrive/signate-908-hiroshima/')
my_dir = os.getcwd()

### 設定

In [ ]:
class Config:
    given_dir = os.path.join(my_dir, 'data', 'given') #ダウンロードしてきたデータのディレクトリ
    data_dir = os.path.join(my_dir, 'data', 'preprocess') #前加工済のデータのディレクトリ
    model_dir = os.path.join(my_dir, 'model') #モデルの格納ディレクトリ
    encoding = 'CP932' #CSV保存時の文字コード
    nrows = None #CSVの読み込み行数　→ 全データの時はNone、試しにコードを動かす時は、10万件などにする
    model_ver = '0_19' #モデルのバージョン番号
    th_per = 0.2 # テスト用データの割合　→20%に設定
    fold = 3 # 交差検証の分割数
    
    # LightGBMのハイパーパラメータを設定
    lgb_params = {
              'objective': 'regression',    # 回帰を指定
              'metric': 'rmse', # 関数はRMSEを使用
              }

my_config = Config()

### 前処理

In [ ]:
# waterlevel前処理用の関数定義

def preprocess_water_data_station(water_data, water_stations):
    """water_dataの観測所名称を修正
    """
    # 欠損値補完
    water_data['river'] = water_data['river'].replace('\u3000', '沼田川')

    # (国)への変更前観測所名を変換
    stations = water_data.loc[water_data['station'].str.contains(r'\(国\)'), 'station'].unique()
    # 中野、伊尾、和木は(国)を含まない観測所が別途存在するため別処理
    stations = [x.replace('(国)', '') for x in stations if x not in ['中野(国)', '伊尾(国)', '和木(国)']]
    water_data['station'] = water_data['station'].apply(lambda x: x + '(国)' if x in stations else x)
    # 中野、伊尾、和木は河川名で分けて処理
    water_data.loc[(water_data['station']=='中野')&(water_data['river']=='太田川'), 'station'] = '中野(国)'
    water_data.loc[(water_data['station']=='伊尾')&(water_data['river']=='芦田川'), 'station'] = '伊尾(国)'
    water_data.loc[(water_data['station']=='和木')&(water_data['river']=='小瀬川'), 'station'] = '和木(国)'

    # (電)への変更前観測所名を変換
    stations = water_data.loc[water_data['station'].str.contains(r'\(電\)'), 'station'].unique()
    stations = [x.replace('(電)', '') for x in stations]
    water_data['station'] = water_data['station'].apply(lambda x: x + '(電)' if x in stations else x)

    # 入力ミスと思われるもの
    water_data['station'] = water_data['station'].replace({'藤波': '藤浪',
                                                           '中州橋': '中洲橋',
                                                           '段原': '段原(猿猴川)'})
    water_data['river'] = water_data['river'].replace({'手越川': '手城川',
                                                       '横川': '横川川'})

    # 入力時使用外の観測所を削除
    in_stations = water_stations.loc[water_stations['入力時使用']==1, '観測所名称'].unique()
    water_data = water_data[water_data['station'].isin(in_stations)]
    water_stations = water_stations[water_stations['入力時使用']==1]

    return water_data, water_stations


def fill_wl_nan(df, station_pair):
    """欠損値を他の観測所の値を使って埋める
    互いに標準化した値でNaNを埋めて、元のスケールに戻す
    """

    for k, v in station_pair.items():
        scaler_k = StandardScaler()
        scaler_v = StandardScaler()
        
        value_k = df.loc[df['station']==k, ['value']]
        value_v = df.loc[df['station']==v, ['value']]

        scaler_k.fit(value_k.values)
        scaler_v.fit(value_v.values)

        value_k = scaler_k.transform(value_k.values)
        value_v = scaler_v.transform(value_v.values)

        value_k[np.isnan(value_k)] = value_v[np.isnan(value_k)]
        df.loc[df['station']==k, ['value']] = scaler_k.inverse_transform(value_k)

    return df

In [ ]:
# waterlevelの読み込み
data_dir = os.path.join(my_config.given_dir, 'waterlevel/data.csv')
data = pd.read_csv(data_dir, encoding='utf8')
data_dir = os.path.join(my_config.given_dir, 'waterlevel/stations.csv')
station_mst = pd.read_csv(data_dir, encoding='utf8')

In [ ]:
# waterlevelの観測所の変更を修正
data, station_mst = preprocess_water_data_station(data, station_mst)

In [ ]:
# waterlevelデータの加工
# 河川名の列は削除
data.drop("river", axis=1, inplace=True)
# 行列を転置させる
data.set_index(['date', 'station'], inplace=True)
data = data.stack().reset_index(drop=False)
data.columns = ["date", "station", "hour", "value"]
# 時間帯をint形式に変換
data["hour"] = data["hour"].str[:2].astype(int)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
# 水位データを数値に変換
data['value'] = pd.to_numeric(data['value'], errors='coerce')
# 明らかな外れ値を修正
data.loc[(data['station']=='山手左岸(国)')&(data['value']<-90), 'value'] = 0
data.loc[data['station']=='七社', 'value'] = data.loc[data['station']=='七社', 'value'].apply(lambda x: round(x / 10, 2) if x > 9 else x)

# NaNを他の観測所のデータから補完
fillnan_pair = {
    '竹の花(国)': '上安田(国)',  # 同水系で相関が強く地理的にも近いもの
    '新市(国)': '上戸手(国)',  # 同水系で相関が強く地理的にも近いもの
    '駅前': '伊尾(国)',
    '上庄(国)': '中深川(国)',
    '市原': '新庄', # 相関が強いもの　候補 '今津', '新庄', '三津', '風早'
    }
data = fill_wl_nan(data, fillnan_pair)

In [ ]:
# 保存する
if os.path.exists(os.path.join(my_config.data_dir, "waterlevel"))==False:
    os.makedirs(os.path.join(my_config.data_dir, "waterlevel"))
    
data.to_csv(os.path.join(my_config.data_dir, "waterlevel", "data.csv"),
            index=False,
            encoding=my_config.encoding)

In [ ]:
# 観測所マスタの作成
# データを読み込む
# 観測所に連番を振っておく
# SJISで保存
# 不要なカラムは削除
station_mst.drop(['フリガナ', "事務所", "データ所管", "住所", "入力時使用"], axis=1, inplace=True)

In [ ]:
# 観測所に連番を振る
station_list = station_mst[['観測所名称']].drop_duplicates()
station_list['station_id'] = range(1, len(station_list.index)+1)
station_list.reset_index(drop=True, inplace=True)
station_mst = pd.merge(station_mst, station_list, on=['観測所名称'], how='left').sort_values('station_id')
# 列名を変更する
station_mst.rename(columns={"河川名":"river", "観測所名称":"station", "市町":"city", "評価対象":'review_tgt'}, inplace=True)

In [ ]:
# 保存する
station_mst.to_csv(os.path.join(my_config.data_dir, 'waterlevel', 'station_master.csv'),
                   encoding=my_config.encoding, index=False)

### クラスを定義

In [ ]:
class ScoringService(object):
    first_flg = True
    # predict時に参照するConfig
    # Configを上書きする時は、get_configを使用する
    class Config:
        nrows = None
        fold = 3
        model_ver = '0_19'
        encoding = 'CP932'
    config = Config()

    @classmethod
    # Configを再取得するメソッド
    def get_config(cls, config):
        try:
            cls.config = config
            return True
        except:
            return False        

    @classmethod
    # モデルを取得するメソッド
    # 交差検証しているため、モデルはリスト(cls.models)に格納される
    def get_model(cls, model_path):
        try:
            cls.models = []
            for i in range(cls.config.fold):
                model = pickle.load(open(os.path.join(model_path,
                    'trained_model_' + cls.config.model_ver + '_fold_' + str(i) + '.pkl'), 'rb'))
                cls.models.append(model)
            return True
        except:
            return False

    @classmethod
    # 予測用のデータを取得する関数
    def get_store_data(cls, data_path):
        try:
            # 観測所マスタと予測対象の読み込み
            cls.station_mst = pd.read_csv(os.path.join(data_path, 'waterlevel', 'station_master.csv'), encoding=cls.config.encoding)
            cls.review_tgt = cls.station_mst.loc[cls.station_mst["review_tgt"]==1,"station"].values
            # 水位データの読み込み
            cls.wl_data = pd.read_csv(os.path.join(data_path, 'waterlevel', 'data.csv'),nrows=cls.config.nrows, encoding='CP932')
            # 評価対象の観測所のみを読み込む
            cls.wl_data = cls.wl_data.loc[cls.wl_data["station"].isin(cls.review_tgt)]
            # 取得したデータを並べ替え、欠損値を埋める
            cls.wl_data['value'] = cls.wl_data['value'].replace({'M':np.nan, '*':np.nan, '-':np.nan, '--': np.nan, '**':np.nan}).astype(float)
            cls.wl_data.sort_values(['station', 'date', 'hour'], inplace=True)
            cls.wl_data['value'] = cls.wl_data.groupby('station')['value'].fillna(method="ffill")
            return True
        except:
            return False

    @classmethod
    # 特徴量作成のメソッドをここに記入する
    def make_feature(cls):
        # df:特徴量のデータフレーム
        cls.df = cls.wl_data[["date", "station", "hour", "value"]].copy()

        # 予測対象時間
        cls.df["予測対象時間"] = cls.df["hour"]

        # 平時の水位
        tmp = pd.DataFrame(cls.df.groupby("station")["value"].median())
        tmp.reset_index(inplace=True)
        tmp.rename(columns={'value':'平時の水位'}, inplace=True)
        cls.df = pd.merge(cls.df, tmp, on = 'station', how='left')

        # 当日同時刻の水位
        cls.df.rename(columns={'value':'当日同時刻の水位'}, inplace=True)
        # 平時からの水位差
        cls.df["当日同時刻の水位_平時差"] = cls.df["当日同時刻の水位"] - cls.df["平時の水位"]

        # 当日23時の水位
        tmp = cls.wl_data.copy()
        tmp = tmp.loc[tmp['hour']== 23,:]
        tmp = tmp[['date', 'station', 'value']]
        tmp.rename(columns={'value':'当日23時の水位'}, inplace=True)
        cls.df = pd.merge(cls.df, tmp, on = ['date', 'station'], how = 'left')

        # 当日0時の水位
        tmp = cls.wl_data.copy()
        tmp = tmp.loc[tmp['hour']== 0,:]
        tmp = tmp[['date', 'station', 'value']]
        tmp.rename(columns={'value':'当日0時の水位'}, inplace=True)
        cls.df = pd.merge(cls.df, tmp, on = ['date', 'station'], how = 'left')
        
        # 当日0時から23時の変化
        cls.df['当日0から23時の変化'] = cls.df['当日23時の水位'] - cls.df['当日0時の水位']
        # 当日0時の水位は削除
        cls.df.drop('当日0時の水位', axis=1, inplace=True)
        
        # staition idを取得
        cls.df = pd.merge(cls.df, cls.station_mst[['station', 'station_id']], how='left', on=['station'])
        return True

    @classmethod
    def train(cls):
        def target_df(df):
            # 翌日同時刻の水位(正解データ)を用意する
            tmp = df.copy()
            tmp['date'] = tmp['date'] - 1
            tmp = tmp [['date', 'station', 'hour', '当日同時刻の水位']]
            tmp.rename(columns={'当日同時刻の水位':'翌日同時刻の水位'}, inplace=True)
            df = pd.merge(df, tmp, on=["date", "station", "hour"], how="left")
            return df
        def train_test_split(df):
            # 学習用データとテスト用データを分割する
            th_date = df["date"].max() - round(df["date"].max() * cls.config.th_per, 0)
            train = df.loc[df['date'] < th_date,:]
            test = df.loc[df['date'] >= th_date,:]
            print("学習用データ：{}日　評価用データ：{}日".format(th_date, df["date"].max()-th_date))
            return train, test
        def sort_and_fillna(df):    
            # 欠損値処理を行う関数
            df.sort_values(["station", "date", "hour"], inplace=True)
            df.dropna(subset = ['翌日同時刻の水位'], inplace=True)
            df['翌日同時刻の水位'] = df['翌日同時刻の水位'].astype(float)
            df = df.groupby('station').apply(lambda x: x.ffill().bfill())
            return df
        def train_model():
            # 学習用の関数
            # dateごとにGroupFoldを行う
            group = cls.train_X['date']
            valid_scores = []
            models = []
            kf = GroupKFold(n_splits=cls.config.fold)

            # GroupFold数だけループする
            for fold, (train_indices, valid_indices) in enumerate(kf.split(cls.train_X, cls.train_y, group)):
                X_train, X_valid = cls.train_X.iloc[train_indices], cls.train_X.iloc[valid_indices]
                y_train, y_valid = cls.train_y.iloc[train_indices], cls.train_y.iloc[valid_indices]
                X_train.set_index(['station', 'date', 'hour'], inplace=True)
                X_valid.set_index(['station', 'date', 'hour'], inplace=True)
                y_train.set_index(['station', 'date', 'hour'], inplace=True)
                y_valid.set_index(['station', 'date', 'hour'], inplace=True)

                lgb_train = lgb.Dataset(X_train.reset_index(drop=True), y_train)
                lgb_valid = lgb.Dataset(X_valid.reset_index(drop=True), y_valid, reference=lgb_train)
                cat_list = ['station_id']
                model = lgb.train(
                        cls.config.lgb_params,
                        lgb_train,
                        categorical_feature = cat_list,
                        valid_sets=lgb_valid,
                        verbose_eval = 50,
                        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)]
                )
                y_valid['predict'] = model.predict(X_valid)
                score = rmse_score(y_valid, d_key1 = '翌日同時刻の水位', d_key2 = 'predict')
                valid_scores.append(score)
                models.append(model)
            cv_score = np.mean(valid_scores)
            print(f"CV score: {cv_score}")
            return models

        def save_model(models):
            # モデルを保存する関数
            for i in range(cls.config.fold):
                file = os.path.join(cls.config.model_dir, 
                                    'trained_model_' + cls.config.model_ver + '_fold_' + str(i) + '.pkl')
                pickle.dump(models[i], open(file, 'wb'))

        def rmse_score(df, d_key1, d_key2):
            # RMSE算出用の自作関数
            tmp = df.copy()
            tmp["diff_"] = tmp[d_key1] - tmp[d_key2]
            tmp["diff_"] = tmp["diff_"].apply(lambda x: x ** 2)
            return np.sqrt(tmp["diff_"].mean())

        # ここからが実際の学習パート
        cls.df = target_df(cls.df)  # 目的変数作成
        train, test = train_test_split(cls.df)  # テストデータを分割
        train = sort_and_fillna(train)  # 
        test = sort_and_fillna(test)

        # 予測対象のカラムを特徴量から削除する
        cls.train_X = train.drop("翌日同時刻の水位", axis=1)
        cls.train_y = pd.DataFrame(train[['station', 'date', 'hour', "翌日同時刻の水位"]])
        cls.test_X = test.drop("翌日同時刻の水位", axis=1)
        cls.test_y = pd.DataFrame(test[['station', 'date', 'hour', "翌日同時刻の水位"]])

        cls.models = train_model()
        save_model(cls.models)
        
        # 予測結果を格納するためのnumpyを用意する
        predict_np = np.zeros([cls.config.fold, len(cls.test_y)])
        predict_np[:,:] = np.nan # いったんnullで埋める
        # Fold分のモデルで予測して中央値を取る
        cls.test_X.set_index(['station', 'date', 'hour'], inplace=True)
        for i in range(cls.config.fold):
            predict_np[i,] = cls.models[i].predict(cls.test_X)
        cls.test_y['predict'] = np.median(predict_np, axis=0)
        cls.test_y.reset_index(inplace=True)

        # 結果を表示する
        tmp = rmse_score(cls.test_y, d_key1 = '翌日同時刻の水位', d_key2 = 'predict')
        print('予測RMSE:{:.4f}'.format(tmp))
        return cls.test_y

    @classmethod
    def predict(cls, input):
        def store_data(input, key=''):
            # 1日毎に与えられるデータを保存する関数
            add_data = pd.DataFrame(input[key])
            add_data['date'] = input['date']
            # データディレクトリに、都度データを格納する
            # ディレクトリがなければ作成する
            my_dir = 'data/' + key
            if not os.path.exists(my_dir):
                os.makedirs(my_dir) 
            # 読み込みファイルがなければ作成する
            if not os.path.exists(os.path.join(my_dir, 'data.csv')) or cls.first_flg == True:
                df = pd.DataFrame(columns=['date', 'hour', 'station', 'value'])
                cls.first_flg = False
            else:
                df = pd.read_csv(os.path.join(my_dir, 'data.csv'), encoding=cls.config.encoding)
            # 読み込んだファイルと合体させて再格納する
            if input['date'] > df['date'].max() or len(df) == 0:
                df = pd.concat([df, add_data], axis = 0)
                df = df.sort_values(['date', 'station', 'hour'])
                # 読み込みファイルサイズの都合上、読み込む日数は過去10日まで
                df = df.loc[df['date'] >= df['date'].max()-10,:].copy()
                df.to_csv(os.path.join(my_dir, 'data.csv'), index=False, encoding=cls.config.encoding)

        def sort_and_fillna(df):    
            df.sort_values(["station", "date", "hour"], inplace=True)
            df = df.groupby('station', group_keys=False).apply(lambda x: x.ffill().bfill())        
            return df

        def model_predict(X):
            # 予測用関数
            # 予測結果を格納するためのnumpyを用意する
            predict_np = np.zeros([cls.config.fold, len(X)])
            predict_np[:,:] = np.nan # いったんnullで埋める
            # Fold分のモデルで予測して中央値を取る
            for i in range(cls.config.fold):
                predict_np[i,] = cls.models[i].predict(X)
            return np.median(predict_np, axis=0)
        def fill_error(df, station_mst):
            # station　×　hour　全通り組み合わせを作成し、提出データに抜け漏れがないようにする
            df1 = pd.DataFrame({'station' : station_mst.loc[station_mst['review_tgt'] == 1, 'station'].unique()})
            df1["dummy"] = 1
            df2 = pd.DataFrame({'hour' : df['hour'].unique()})
            df2["dummy"] = 1
            dummy_df = pd.merge(df1, df2, on='dummy', how='outer')[['station', 'hour']]
            df = pd.merge(df, dummy_df, on = ['station', 'hour'], how = 'outer')
            return df.fillna(0)

        # データをデータフレーム形式で格納する
        store_data(input, key = 'waterlevel')

        # モデルを取得する
        cls.get_model(os.path.join('..', 'model'))
        # データを取得する
        _ = cls.get_store_data('data')
        # データを予測する
        cls.make_feature()
        cls.df = sort_and_fillna(cls.df)
        # 生成された特徴量DFのうち、当該日のdateのみを抽出する
        cls.df = cls.df.loc[cls.df['date'] == input['date']]
        predict = cls.df.copy()        
        predict.set_index(['date', 'station', 'hour'], inplace=True)
        predict['value'] = model_predict(predict)
        predict.reset_index(inplace=True)

        # エラー予防処理の実施
        predict = fill_error(predict, cls.station_mst)
        predict = predict[["station", "hour", "value"]]
        prediction = predict.to_dict('records')
        return prediction

In [ ]:
ss = ScoringService
ss.get_config(my_config)
ss.get_store_data(my_config.data_dir)
ss.make_feature()
ss.train()

学習用データ：1752.0日　評価用データ：438.0日


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-37-26228431de04>:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['翌日同時刻の水位'] = df['翌日同時刻の水位'].astype(float)
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1208: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['station_id']
  warnings.warn('categorical_feature in Dataset is overridden.\n'
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:762: Us

Training until validation scores don't improve for 10 rounds.
[50]	valid_0's rmse: 0.601814
Early stopping, best iteration is:
[56]	valid_0's rmse: 0.598945


<ipython-input-37-26228431de04>:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_valid['predict'] = model.predict(X_valid)


Training until validation scores don't improve for 10 rounds.
[50]	valid_0's rmse: 0.645898
Early stopping, best iteration is:
[75]	valid_0's rmse: 0.635065
Training until validation scores don't improve for 10 rounds.
[50]	valid_0's rmse: 0.302608
[100]	valid_0's rmse: 0.271366
Did not meet early stopping. Best iteration is:
[91]	valid_0's rmse: 0.270886
CV score: 0.5016319190736506
予測RMSE:0.1034


,index,station,date,hour,翌日同時刻の水位,predict
0,42048,七宝,1752,0,1.71,1.678519
1,42049,七宝,1752,1,1.72,1.678519
2,42050,七宝,1752,2,1.73,1.678519
3,42051,七宝,1752,3,1.72,1.675341
4,42052,七宝,1752,4,1.70,1.668540
...,...,...,...,...,...,...
1744987,8728915,黒滝(国),2189,19,0.24,0.455082
1744988,8728916,黒滝(国),2189,20,0.24,0.459495
1744989,8728917,黒滝(国),2189,21,0.24,0.459495
1744990,8728918,黒滝(国),2189,22,0.24,0.443375
